# A bit of review on data structures

- Data structures are built in to Clojure
- Lots of (hundreds) of functions to access and transform data in Clojure.
- Consider Clojure has a data analysis language that can do everything else.

In [2]:
;; Let's build a dictionary
;; and bind it to a global symbol

(def grades {:mary 90
            :joe 80
            :jack 70})

#'user/grades

# Task 1: find the best student and the worst student

1. Transform the dictionary into a list
2. Sort the list by the grade
3. Get the first and the last elements of the sorted list. 

_Getting a bit head of ourselves..._

In [5]:
; Make it into a list of tuples
(for [[k v] grades] [v k])

([90 :mary] [80 :joe] [70 :jack])

In [7]:
; Sort them - the default sorting is lexicographical
; ordering, namely sort by first coordinate.
(sort (for [[k v] grades] [v k]))

([70 :jack] [80 :joe] [90 :mary])

In [8]:
; Now we can get the first and last
(let [list (sort (for [[k v] grades] [v k]))]
  {:best (first list)
   :worst (last list)})

{:best [70 :jack], :worst [90 :mary]}

In [9]:
;; Ooops, the sorting should be descending
;; Let's define our own comparator
;; Note that we are jusing a function
;; value: (fn [x y] ...) which reverses
;; the comparison returned by (compare ...)
(sort (fn [x y] (* -1 (compare x y)))
      (for [[k v] grades] [v k]))

([90 :mary] [80 :joe] [70 :jack])

In [11]:
(defn my-cmp [x y] (* -1 (compare x y)))

#'user/my-cmp

In [12]:
(let [list (sort my-cmp (for [[k v] grades] [v k]))]
  {:best (first list)
   :worst (last list)})

{:best [90 :mary], :worst [70 :jack]}

In [13]:
;; Now I just want their names, not their grades.
(let [list (sort my-cmp (for [[k v] grades] [v k]))]
  {:best (get (first list) 1)
   :worst (get (last list) 1)})

{:best :mary, :worst :jack}

In [17]:
;; Shorter solution to the task
(-> (for [[k v] grades] [v k])
    (sort)
    (reverse)
    (first)
    (get 1))

:mary

# End of task 1

# Let's look at some elements of programming constructs we have used

In [14]:
;; Defining functions
;; Functions are JUST values like numbers
;; If you want to use a function more than once,
;; you need to bind it to a symbol.
;; Don't forget about the scoping of your binding.

In [20]:
(defn sum-of-squares [x y z]
  (+ (* x x) (* y y) (* z z)))

#'user/sum-of-squares

In [23]:
; Try it out
(sum-of-squares 1 2 3)

14

In [25]:
((fn [x y z] (+ (* x x) (* y y) (* z z))) 1 2 3)

14

In [26]:
(def sum-of-square (fn [x y z] (+ (* x x) (* y y) (* z z))))

#'user/sum-of-square

In [27]:
(sum-of-squares 1 2 3)

14

# Hard way 1: Recursion

In [37]:
;; sos = sum-of-squares
(defn sos-1 [& numbers]
  (cond
    (empty? numbers) 0
    ;; now recursion
    :else (let [x (first numbers)
                n (rest numbers)]
             (+ (* x x) (apply sos-1 n)))))

#'user/sos-1

In [41]:
;; DOES IT WORK???
(sos-1 1 2 3 4 5 6)

91

# Hard way 2: Tail recursion

Clojure cares about performance.  Recursion is the most expensive way of iterating
over a collection.  Tail-recursion is a special form that can be optimized into
a native JVM optimized for-loop, which is A LOT FASTER.  We will see an example later.

The lecture notes talk about this form of recursion using the `Iteration with loop and recur` construct.

In [43]:
(defn sos-2 [& numbers]
  (loop [result 0
         n      numbers]
    (if (empty? n)
      result
      (let [x (first n)]
        (recur (+ (* x x) result) (rest n))))))

#'user/sos-2

In [44]:
(sos-2 1 2 3 4 5)

55

# Easy way: using built-in functions

In [45]:
(defn sos-3 [& numbers]
  (apply + (map (fn [x] (* x x)) numbers)))

#'user/sos-3

In [47]:
(sos-3 1 2 3 4 5)

55

In [48]:
(apply + (map (fn [x] (* x x)) [1 2 3 4 5]))

55

# Easy way 2: reduction

In [49]:
(defn accumulator [result x] (+ result (* x x)))

#'user/accumulator

In [50]:
(reduce accumulator 0 [1 2 3 4 5])

55

In [51]:
(defn sos-3 [& numbers]
  (reduce accumulator 0 numbers))

#'user/sos-3

In [52]:
(sos-3 1 2 3 4 5 6 7 8 9 10)

385